
<div style="background-color:#CCCCCC; padding:12px; border-radius:8px;">
<h1 style="color:#003366; text-align:center; margin:8px 0;">Revisión y limpieza de 3 DataFrames (TPI - Data Analytics)</h1>
<p style="text-align:center; color:#003366; margin:0;"><em>Notebook docente en castellano — nombres descriptivos en snake_case — código y documentación</em></p>
</div>



<div style="background-color:#CCCCCC; padding:10px; border-radius:6px;">
<h2 style="color:black; text-align:center; margin-top:6px;">Resumen</h2>

<p style="color:black;">
Este notebook está diseñado con finalidades pedagógicas. Revisa, normaliza y valida tres datasets contenidos en CSV:
</p>

<ul style="color:black;">
<li><code>marketing.csv</code> → variable: <code>df_marketing</code></li>
<li><code>ventas.csv</code>    → variable: <code>df_ventas</code></li>
<li><code>clientes.csv</code>  → variable: <code>df_clientes</code></li>
</ul>

<p style="color:black;">
Coloca los CSV en <code>./data_in/</code> o en <code>/mnt/data/</code>. El notebook busca primero en <code>./data_in/</code> y si no encuentra, usa <code>/mnt/data/</code> (útil para entornos donde los archivos están pre-subidos).
</p>

</div>


In [1]:
# Imports y configuración inicial (nombres en castellano)
import os
from pathlib import Path
import json
import unicodedata
from colorama import *
import re
#!pip install gdown

import argparse
import pandas as pd
import numpy as np
from typing import Dict, Any, Optional
import zipfile
from collections import defaultdict
from datetime import datetime
from math import isnan
ruta_base = ""


## 1. Crear un documento en Google Colaboratory y cargar los sets de datos como DataFrames

si se usa en disco local comentarla celda de debajo (JuPyteR , VSC, ATOM, Spider, Geany, etc)

# 1ra parte Definición de ETL
ETL es un conjunto de procedimientos que permiten mover datos desde sistemas de origen, que pueden ser bases de datos, archivos o fuentes en la nube, hasta un sistema de destino como un data warehouse o data lake, realizando previamente procesos de limpieza, estructuración y organización de los datos para hacerlos aptos para análisis.​

## Fases del proceso ETL
Extracción: Consiste en recopilar datos relevantes de diferentes fuentes, asegurando que el impacto en los sistemas origen sea mínimo. Los datos pueden extraerse mediante diversos métodos como consultas SQL o servicios web.​

Transformación: En esta etapa, los datos se limpian y se ajustan para garantizar coherencia y calidad, incluyendo la eliminación de valores nulos, normalización y conversión a formatos consistentes, además de aplicar reglas específicas de negocio.

Carga: Finalmente, los datos transformados se cargan en el sistema de destino, donde estarán disponibles para análisis, informes o modelado de datos.

## Importancia del ETL
Es crucial en la minería de datos porque preparar los datos brutos para que puedan ser utilizados en análisis estadísticos, modelados predictivos o técnicas de aprendizaje automático, asegurando la calidad, coherencia y accesibilidad de la información.

## desde python sin librerias pandas / polars

### 1.1 Crear estructura de directorios segun modelo

In [2]:
# Rutas: 

carpeta_entrada    = Path(ruta_base)
#carpeta_entrada_mnt   = Path('/mnt/data')
carpeta_datasets_entrada   = carpeta_entrada / 'datasets_entrada'
carpeta_datasets_entrada.mkdir(parents=True, exist_ok=True)

carpeta_datasets_salida   = carpeta_entrada / 'datasets_salida'
carpeta_datasets_salida.mkdir(parents=True, exist_ok=True)

carpeta_reportes   = carpeta_datasets_salida / 'reportes'
carpeta_reportes.mkdir(parents=True, exist_ok=True)

carpeta_limpios    = carpeta_datasets_salida / 'limpios'
carpeta_limpios.mkdir(parents=True, exist_ok=True)


# Nombres esperados de archivos
archivo_ventas     = 'ventas.csv'
archivo_clientes   = 'clientes.csv'
archivo_marketing  = 'marketing.csv'

### 1.2 rutas y carga de los dataframes

In [3]:
# --- Paso 3: Cargar archivos del curso ---
print("Cargando datasets del curso...")
try:
    ruta_ventas     = os.path.join(carpeta_datasets_entrada, archivo_ventas)
    ruta_clientes   = os.path.join(carpeta_datasets_entrada, archivo_clientes)
    ruta_marketing  = os.path.join(carpeta_datasets_entrada, archivo_marketing)
    
    df_ventas       = pd.read_csv(f"{ruta_ventas}")
    df_clientes     = pd.read_csv(f"{ruta_clientes}")
    df_marketing    = pd.read_csv(f"{ruta_marketing}")
    dic_dfs = { "df_ventas"   : df_ventas,
                "df_clientes" : df_clientes,
                "df_marketing": df_marketing}
    print ("...Arhivos cargados")
except:
    print ("Arhivos no encontrados")
    exit()

Cargando datasets del curso...
...Arhivos cargados


### 1.3 Estructura de parámetros 

In [39]:
# ---------- Proceso principal para los 3 CSV ----------
desviacion_margen     = 1.5
desviacion_umbral     = 3.0
cantidad_duplicados   = 0
reportes_creados      = []
ruta_excel            = carpeta_reportes / 'reporte_limpieza.xlsx'
guardado_ok           = False
mensajes              = []
TOKENS_VALOR_FALTANTE = {'na', 'n/a', 'null', 'none', 'sin dato', 's/d', 'nd', '-', '--', '?', 'sin_dato', 'n/d'}
reglas= {
            # id_campanha, producto, canal, costo, fecha_inicio, fecha_fin
            "df_marketing" : {
                                'producto':     ('string',  {'tipo': 'lower', 'normalizar_acentos': True}),
                                'canal':        ('string',  {'tipo': 'upper', 'normalizar_acentos': True}),
                                'costo':        ('numeric', {'as_int': False}),
                                'fecha_inicio': ('date',    {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']}),
                                'fecha_fin':    ('date',    {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']})
                                },
            # id_venta, producto, precio, cantidad, fecha_venta, categoria
            "df_ventas" : {
                                'producto':     ('string',  {'tipo': 'lower', 'normalizar_acentos': True}),
                                'precio':       ('numeric', {'as_int': False}),
                                'cantidad':     ('numeric', {'as_int': False}),
                                'fecha_venta':  ('date',    {'dayfirst': True, 'formats': ['%d/%m/%Y', '%Y-%m-%d']}),
                                'categoria':    ('string',  {'tipo': 'lower', 'normalizar_acentos': True})
                            },
            # id_cliente, nombre, edad, ciudad, ingresos
            "df_clientes" : {
                                'nombre':       ('string',  {'tipo': 'title', 'normalizar_acentos': True}),
                                'edad':         ('numeric', {'as_int': True}),
                                'ciudad':       ('string',  {'tipo': 'title', 'normalizar_acentos': True}),
                                'ingresos':     ('numeric', {'as_int': False})
                            }
    }
reglas_por_archivo = {
    'ventas.csv':    reglas_ventas,
    'clientes.csv':  reglas_clientes,
    'marketing.csv': reglas_marketing
}

#zip_path = carpeta_reportes.parent / 'reports_dataset_tpi_v2.zip'


In [40]:
# Mostrar el DataFrame
def ver():
    for nombre,df in dic_dfs.items():
        print(f"""
    {nombre}
        Descripción preliminar:
        {df.describe()}
        Dimensiones:{ df.ndim}
        Forma:{ df.shape}    
        Número de elementos:{ df.size}
        Nombres de columnas:{ df.columns}
        Nombres de filas:{ df.index}
        Tipos de datos:\n{ df.dtypes}
        Primeras 10 filas:\n{ df.head(10)}
        Últimas 3 filas:\n{ df.tail(3)}
    {"*"*50}
    """)
ver()


    df_ventas
        Descripción preliminar:
                  id_venta     cantidad
count  3035.000000  3033.000000
mean   1499.851400     6.496538
std     866.465379     3.457250
min       1.000000     1.000000
25%     748.500000     3.000000
50%    1502.000000     7.000000
75%    2249.500000     9.000000
max    3000.000000    12.000000
        Dimensiones:2
        Forma:(3035, 6)    
        Número de elementos:18210
        Nombres de columnas:Index(['id_venta', 'producto', 'precio', 'cantidad', 'fecha_venta',
       'categoria'],
      dtype='object')
        Nombres de filas:RangeIndex(start=0, stop=3035, step=1)
        Tipos de datos:
id_venta         int64
producto        object
precio          object
cantidad       float64
fecha_venta     object
categoria       object
dtype: object
        Primeras 10 filas:
   id_venta           producto   precio  cantidad fecha_venta  \
0       792  Cuadro decorativo   $69.94       5.0  02/01/2024   
1       811    Lámpara de mesa  $105.

In [41]:
# ---------- Funciones utilitarias en castellano (snake_case) ----------
def quitar_acentos(entrada):
    """
    Elimina acentos (tildes/diacríticos) de un texto.
    Mantiene NaN intactos.
    """
    if pd.isna(entrada):
        return entrada
    if isinstance(entrada, str):
        #nk = unicodedata.normalize('NFKD', entrada)
        #salida = ''.join([c for c in nk if not unicodedata.combining(c)])
        texto_n = unicodedata.normalize("NFD", entrada)
        return "".join(c for c in texto_n if unicodedata.category(c) != "Mn")
    return entrada
    '''
    if salida != entrada:
        print("""\033[1;37;44m\n
╔═════════════════════════════════════════════════════════════════════════════╗
║                                sacar_acentos                                ║
╚═════════════════════════════════════════════════════════════════════════════╝\033[0;m""")
    print(f"    {texto} -->  {salida}")
    '''
    return salida

In [42]:
def to_num(serie):
    """
    Convierte una serie a tipo numérico.
    - Elimina símbolos como $, % y comas.
    - Reemplaza coma decimal por punto.
    - Si no puede convertir, deja NaN.
    """
    salida =  (serie.astype(str)
                  .str.replace(r"[^0-9,.\-]", "", regex=True)
                  .str.replace(",", ".", regex=False)
                  .replace("", np.nan)
                  .astype(float))

    
    return salida



    

In [ ]:
def limpiar_dataframes() :
    """
    Limpia el DataFrame aplicando reglas_por_columna = {"col": ("regla", parametros)...}
    Las reglas se asignan automáticamente según el tipo o formato:
      - Columnas numéricas o con símbolos ($, %, dígitos) → 'numeric'
      - Columnas que parecen fechas → 'date'
      - Otras columnas → 'string'
    """



    for nombre_df, df_cada in dic_dfs.items():
        print (f"""{nombre_df}
        {dic_dfs[nombre_df]}
        """)
        
        for nombre_columna, serie in df_cada.items():
            nombre_columna= nombre_columna.lower().replace(" ","_")
            if nombre_columna.startswith("id_"):
                continue
            regla, params = reglas[nombre_df][nombre_columna]]
            print (f"""
            {nombre_columna=}
            {serie=}
            {regla=}
            """)
            s = serie.copy()
            if regla == "numeric":
                s = s.strip()
                s = s.replace('$', '')
                s = pd.to_numeric(s, errors="coerce")
                if regla["numeric"]["as_int"] :
                    s = s.replace('.', '').replace(',', '')
                    if not s.isna().any():
                        s = s.astype(int)
            elif regla == "string":
                s = s.astype(str).str.string()
                match  regla["numeric"]["tipo"] :
                    case "upper":
                        s = s.str.upper()
                    case "title":
                        s = s.str.title()
                    case "lower":
                        s = s.str.lower()
            elif regla == "date":
                if regla["date"]["dayfirst"]:
                    salida = serie.dt.strftime('%Y/%m/%d', errors="coerce", dayfirst=True)
                    #'formats': ['%d/%m/%Y', '%Y-%m-%d']}),
                else:
                    salida = serie.dt.strftime('%Y/%m/%d', errors="coerce", dayfirst=False)                
            elif regla == "fillna":
                s = s.fillna(0)
            dic_dfs[nombre_df][nombre_columna] = s
    
            
            #params = regla_info[1] if len(regla_info) > 1 else None
            #df[col] = aplicar_regla_columna(serie, regla, formatos=params)
            print (f"""
            {"*"*50}
            {regla=}""")
        print (dic_dfs[nombre_df].head(40))
limpiar_dataframes()

df_ventas
              id_venta                producto   precio  cantidad fecha_venta  \
0          792       Cuadro decorativo   $69.94       5.0  02/01/2024   
1          811         Lámpara de mesa  $105.10       5.0  02/01/2024   
2         1156                Secadora   $97.96       3.0  02/01/2024   
3         1372                Heladera  $114.35       8.0  02/01/2024   
4         1546                Secadora  $106.21       4.0  02/01/2024   
...        ...                     ...      ...       ...         ...   
3030      1837         Horno eléctrico  $104.12       9.0  30/12/2024   
3031      2276                  Laptop   $85.27       9.0  30/12/2024   
3032      2696                  Laptop  $107.81       4.0  30/12/2024   
3033      2913              Smartphone   $99.85       7.0  30/12/2024   
3034      2930  Consola de videojuegos   $55.47       6.0  30/12/2024   

              categoria  
0            Decoración  
1            Decoración  
2     Electrodomésticos  
3

In [12]:
def es_valor_faltante(valor):
    """
    Determina si un valor debe considerarse faltante (True) usando tokens y NaN.
    """
    if pd.isna(valor):
        return True
    s = str(valor).strip().lower()
    s = sacar_acentos(s)
    return s in TOKENS_VALOR_FALTANTE

In [15]:

# Detección de outliers (IQR) - función corregida y robusta
def mascara_valores_atipicos_rango_intercuartil(serie_datos):
    """
    Devuelve una tupla: (mascara_bool_series, cantidad_outliers, (limite_inferior, limite_superior))
    - serie_datos: pd.Series (acepta valores no numéricos, se intentará convertir)
    - La máscara tiene la misma indexación que la serie original (NaNs -> False)
    """
    # Intentar convertir a numérico (coerce -> NaN para no numéricos)
    serie_numerica = pd.to_numeric(serie_datos, errors='coerce')
    # Serie limpia para cálculos de cuartiles (sin NaN)
    serie_limpia = serie_numerica.dropna().astype(float)
    if serie_limpia.shape[0] < 4:
        # No hay suficientes datos para IQR: devolver máscara False de la misma longitud
        mascara = pd.Series([False] * len(serie_datos), index=serie_datos.index)
        return mascara, int(mascara.sum()), (None, None)
    cuartil_1 = float(serie_limpia.quantile(0.25))
    cuartil_3 = float(serie_limpia.quantile(0.75))
    rango_intercuartil = cuartil_3 - cuartil_1
    limite_inferior = cuartil_1 - desviacion_margen * rango_intercuartil
    limite_superior = cuartil_3 + desviacion_margen * rango_intercuartil
    # Crear máscara sobre la serie numérica original (alineada con el index original)
    mascara = (serie_numerica < limite_inferior) | (serie_numerica > limite_superior)
    mascara = mascara.fillna(False).astype(bool)
    return mascara, int(mascara.sum()), (limite_inferior, limite_superior)

In [16]:
# Detección de outliers (Z-score)
def mascara_valores_atipicos_zscore(serie_datos, desviacion_umbral=3.0):
    """
    Devuelve máscara booleana (True = outlier) según Z-score.
    """
    serie_limpia = serie_datos.dropna().astype(float)
    if serie_limpia.shape[0] < 4 or serie_limpia.std() == 0:
        return pd.Series([False] * len(serie_datos), index=serie_datos.index)
    puntaje_z = (serie_datos - serie_limpia.mean()) / serie_limpia.std()
    return puntaje_z.abs() > desviacion_umbral  
print('Funciones utilitarias definidas.')

Funciones utilitarias definidas.


# limpieza y normalización

In [17]:
# ---------- Detección de problemas en un DataFrame ----------
def detectar_problemas_en_dataframe(df: pd.DataFrame):
 
    resumen                       = {}
    resumen['filas']              = df.shape[0]
    resumen['columnas']           = df.shape[1]
    resumen['nulos_por_columna']  = df.isna().sum().to_dict()
    dup_mask                      = df.duplicated(keep=False)
    resumen['duplicados_exactos'] = int(dup_mask.sum())
    chequeos_por_columna          = {}
    print(f"""\033[1;37;44m\n
╔═════════════════════════════════════════════════════════════════════════════╗
║                              Valores a eliminar                             ║
╠═════════════════════════════════════════════════════════════════════════════╣
║     Afecta                                                                  ║
║         Elimina espacios iniciales y finales.                               ║
║         Borra Na                                                            ║
║         Borra duplicados                                                    ║
╚═════════════════════════════════════════════════════════════════════════════╝\033[0;m""") 
    print(f"""\033[1;37;44m\n
╔═════════════════════════════════════════════════════════════════════════════╗
║                               Valores atípicos                              ║
╠═════════════════════════════════════════════════════════════════════════════╣
║     Afecta                                                                  ║
║         NO Modifica datos.                                                  ║
║         Se guarda la información en archivo excel para referencias futuras  ║
║         Se evalua es mediante dos formas                                    ║
║            1) limites intercuartiles 25 y 75 % * desviacion_margen {desviacion_margen}      ║
║            2) Z-score mayor a desviacion_umbral {desviacion_umbral}                         ║
╚═════════════════════════════════════════════════════════════════════════════╝\033[0;m""")
    for col in df.columns:
        serie = df[col]
        info = {'dtype': str(serie.dtype), 'nulos': int(serie.isna().sum())}
        if serie.dtype == object or pd.api.types.is_string_dtype(serie):
            s = serie.astype(str)
            info['espacios_inicio']          = int(s.str.match(r'^\s+').sum())
            info['espacios_final']           = int(s.str.match(r'\s+$').sum())
            try:
                unique_original              = set(s.dropna().unique())
                unique_lower                 = set(s.dropna().str.lower().unique())
                info['unique_original']      = len(unique_original)
                info['unique_lower']         = len(unique_lower)
                info['variantes_mayusculas'] = len(unique_lower) < len(unique_original)
            except Exception:
                info['unique_original']      = serie.nunique(dropna=True)
                info['unique_lower']         = None
                info['variantes_mayusculas'] = None
            try:
                unaccented                   = s.dropna().map(lambda x: sacar_acentos(x).lower())
                groups                       = unaccented.groupby(unaccented).size()
                conflicts                    = groups[groups > 1]
                info['grupos_var_acentos']   = int(conflicts.shape[0])
                ejemplos = {}
                if not conflicts.empty:
                    for val in conflicts.index[:5]:
                        originales           = sorted(list(s[unaccented == val].unique())[:10])
                        ejemplos[val]        = originales
                info['ejemplos_var_acentos'] = ejemplos
            except Exception:
                info['grupos_var_acentos']   = None
                info['ejemplos_var_acentos'] = {}
            info['tokens_aparente_faltante'] = int(
                s.map(lambda x: str(x).strip().lower()).map(lambda v: sacar_acentos(v) in TOKENS_VALOR_FALTANTE).sum()
            )
            info['muestras'] = list(s.dropna().unique()[:10])
        else:
            # numeric
            if pd.api.types.is_numeric_dtype(serie) or (serie.dropna().astype(str).str.replace('.','',1).str.isnumeric().all() if len(serie.dropna())>0 else False):
                try:
                    serie_numerica = serie.dropna().astype(float)
                except Exception:
                    serie_numerica = pd.to_numeric(serie, errors='coerce').dropna().astype(float)
                info['media'] = float(serie_numerica.mean()) if not serie_numerica.empty else None
                info['std']   = float(serie_numerica.std()) if not serie_numerica.empty else None
                info['min']   = float(serie_numerica.min()) if not serie_numerica.empty else None
                info['max']   = float(serie_numerica.max()) if not serie_numerica.empty else None
                if len(serie_numerica) >= 4:
                    mascara_outliers_iqr, cant,(info['outliers_iqr'] ,info['limites_iqr']) =mascara_valores_atipicos_rango_intercuartil(serie)
                    #mascara, int(mascara.sum()), (limite_inferior, limite_superior)
                    '''
                    print (f"""
                    {mascara_outliers_iqr=}
                    {cant=}
                    {info['outliers_iqr']=}
                    {info['limites_iqr']=}
                    {"-"*100}
                    """)
                    '''
                    
                else:
                    info['outliers_iqr'] = None
                    info['limites_iqr'] = None
                if len(serie_numerica) >= 4 and serie_numerica.std() != 0:
                    z = (serie_numerica - serie_numerica.mean()) / serie_numerica.std()
                    info['outliers_z'] = int((z.abs() > 3).sum())
                else:
                    info['outliers_z'] = None
            else:
                # fechas intento parseo
                parsed = pd.to_datetime(serie, errors='coerce', dayfirst=True)
                info['fechas_parseables'] = int(parsed.notna().sum())
                info['muestras'] = list(serie.dropna().unique()[:10])
        chequeos_por_columna[col] = info

    filas_problemas = []
    df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
    for idx, fila in df.iterrows():
        lista_problemas = []
        if dup_mask.loc[idx]:
            lista_problemas.append('duplicado_exacto')
        for col in df.columns:
            val = fila[col]
            
            # heurísticas textuales
            if pd.api.types.is_string_dtype(type(val)) or isinstance(val, str) or (
                not pd.isna(val) and not pd.api.types.is_numeric_dtype(type(val)) and str(chequeos_por_columna[col].get('dtype','')).startswith('object')
            ):
                s = str(val)
                if s != s.strip():
                    lista_problemas.append(f'espacios_en_columna_{col}')
                if chequeos_por_columna[col].get('variantes_mayusculas'):
                    if s and s != s.lower() and s.lower() in [str(x).lower() for x in df[col].dropna().unique()]:
                        lista_problemas.append(f'inconsistencia_mayusculas_columna_{col}')
                if chequeos_por_columna[col].get('grupos_var_acentos') and chequeos_por_columna[col]['grupos_var_acentos'] > 0:
                    try:
                        #un = sacar_acentos(s).lower()
                        #group_vals = [x for x in chequeos_por_columna[col].get('muestras', []) if sacar_acentos(str(x)).lower() == un]
                        
                        
                        
                        
                        df['col'] = df['col'].apply(sacar_acentos)
                        
                        
                        
                        if group_vals and any(sacar_acentos(str(x)).lower() != sacar_acentos(s).lower() for x in group_vals):
                            lista_problemas.append(f'variantes_acentos_columna_{col}')
                    except Exception:
                        pass
                if es_valor_faltante(s):
                    lista_problemas.append(f'token_faltante_columna_{col}')
            else:
                # heurísticas numéricas
                try:
                    fval = float(val)
                    info_col = chequeos_por_columna[col]
                    limites = info_col.get('limites_iqr')
                    if limites and (fval < limites[0] or fval > limites[1]):
                        lista_problemas.append(f'outlier_iqr_columna_{col}')
                    if info_col.get('std') not in (None, 0):
                        mean = info_col.get('media')
                        std = info_col.get('std')
                        if std and abs((fval - mean) / std) > 3:
                            lista_problemas.append(f'outlier_z_columna_{col}')
                except Exception:
                    pass
        if lista_problemas:
            filas_problemas.append({
                'row_index': idx,
                'problemas': ';'.join(sorted(set(lista_problemas))),
                'muestra': json.dumps({str(c): str(fila[c]) for c in df.columns[:8]})
            })

    df_problemas = pd.DataFrame(filas_problemas)
    return resumen, chequeos_por_columna, df_problemas

print('Función detectar_problemas_en_dataframe cargada.')

Función detectar_problemas_en_dataframe cargada.


In [18]:
# Función auxiliar para guardar CSVs
def guardar_csv(df, ruta):
    """
    Guarda df en ruta (string o Path). Crea directorio padre si no existe.
    """
    ruta = Path(ruta)
    ruta.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(ruta, index=False, encoding='utf-8')
    return ruta

In [19]:
dic_frames = {
    ruta_ventas     : df_ventas,
    ruta_clientes   : df_clientes,
    ruta_marketing  : df_marketing
}

In [20]:
# ---------- Bucle/menu principal (usa dic_frames) ----------
def menu_procesar_diccionario(dic_frames, reglas_por_archivo):
    """
    Recorre dic_frames: clave = nombre_archivo (ej. 'marketing.csv'), valor = DataFrame.
    Ejecuta: detectar_problemas_en_dataframe antes, limpiar_dataframe, detectar_problemas_en_dataframe despues,
    imprime resúmenes y guarda cleaned en carpeta_limpios con sufijo ' limpio.csv'.
    También sobreescribe variables en RAM (df_marketing, df_ventas, df_clientes) si se encuentran en el nombre.
    """
    print("""\033[1;37;44m\n
╔═════════════════════════════════════════════════════════════════════════════╗
║                     Aplico reglas según columna específica                  ║
╠═════════════════════════════════════════════════════════════════════════════╣
║     Afecta                                                                  ║
║             Numéricos (int/float)                                           ║
║             fechas --> YYYY,MM,DD                                           ║
╚═════════════════════════════════════════════════════════════════════════════╝\033[0;m""")
    errores_df=pd.DataFrame()
    # trabajamos sobre una copia para evitar modificar dict original por error
    for [path_archivo, df_actual],[nombre_archivo,_] in zip( dic_frames.items() , reglas_por_archivo.items() ):
        resumen_antes, chequeos_antes, problemas_antes = detectar_problemas_en_dataframe(df_actual)
        print(f"--- RESUMEN ANTES: {nombre_archivo} ---")
        # Para no volcar objetos muy grandes, mostramos el head del DataFrame de problemas (si existe)
        #print('Muestras de problemas antes (primeras 5 filas):')
        #print( display(problemas_antes.head(5) if not problemas_antes.empty else 'No se detectaron filas con problemas.') )

        errores_df  = pd.concat ([errores_df,problemas_antes])

        #df_actual[df_actual.duplicated(keep=False)].copy()
        reglas    = reglas_por_archivo.get(nombre_archivo, {})
        df_limpio = limpiar_dataframe(df_actual, reglas_por_columna=reglas)

        resumen_despues, chequeos_despues, problemas_despues = detectar_problemas_en_dataframe(df_limpio)
        '''
        print(f"n--- RESUMEN DESPUÉS: {nombre_archivo} ---")
        print(resumen_despues)
        print('Muestras de problemas después (primeras 5 filas):')
        print( display(problemas_despues.head(5) if not problemas_despues.empty else 'No se detectaron filas con problemas tras la limpieza.'))
        '''
        # mostrar separadores y tipo-nombre

        #print('-'*100)
        #print(f'nombre_archivo = {nombre_archivo}')
        #print(f'type(nombre_archivo) = {type(nombre_archivo)}')
        #print('-'*100)

        # Guardo el archivo en limpios  ruta_base carpeta_reportes
        nombre_limpio = nombre_archivo[:-4] + '_limpio.csv' if nombre_archivo.lower().endswith('.csv') else nombre_archivo + ' limpio.csv'
        print(f'nombre_limpio = {nombre_limpio}')
    
        # guardar cleaned
        ruta_guardado = carpeta_limpios / nombre_limpio
        guardar_csv(df_limpio, ruta_guardado)
        print(f'Guardado cleaned en: {ruta_guardado}')

        # sobreescribir en RAM según el nombre
        # (nota: usar globals() para actualizar variables en el entorno global del notebook)
        dic_dfs[nombre_archivo.lower()] = df_limpio
        '''
        if 'marketing' in nombre_archivo.lower():
            globals()['df_marketing'] = df_limpio
        elif 'ventas' in nombre_archivo.lower():
            globals()['df_ventas'] = df_limpio
        elif 'clientes' in nombre_archivo.lower():
            globals()['df_clientes'] = df_limpio
        '''
    print('Proceso completo del diccionario de DataFrames.')
    return errores_df
errores_df = menu_procesar_diccionario(dic_frames, reglas_por_archivo)




╔═════════════════════════════════════════════════════════════════════════════╗
║                     Aplico reglas según columna específica                  ║
╠═════════════════════════════════════════════════════════════════════════════╣
║     Afecta                                                                  ║
║             Numéricos (int/float)                                           ║
║             fechas --> YYYY,MM,DD                                           ║
╚═════════════════════════════════════════════════════════════════════════════╝


╔═════════════════════════════════════════════════════════════════════════════╗
║                              Valores a eliminar                             ║
╠═════════════════════════════════════════════════════════════════════════════╣
║     Afecta                                                                  ║
║         Elimina espacios iniciales y finales.                               ║
║         Borra Na                  

NameError: name 'sacar_acentos' is not defined

<div style="background-color:#CCCCCC; padding:10px; border-radius:6px;">
<h2 style="color:black; text-align:center;">Resultados de limpieza</h2>
<p style="color:black;">- Revisados los 3 csv  pasados a DataFrames.</p>
<p style="color:blue;">- DataFrames filtrados.</p>
<p style="color:black;">- Filtrado de Nulos.</p>
<p style="color:black;">- Filtrado de duplicados.</p>
<p style="color:black;">- Sin '', 'na', 'n/a', 'null', 'none', 'sin dato', 's/d', 'nd', '-', '--', '?', 'sin_dato', 'n/d'</p>    
<p style="color:black;">- Normalisados Strings segun reglas. Estilo (lower,string.upper) unicodedata.normalize('NFKD')</p>
<p style="color:black;">- Normalisados precios a float sin signo ($)</p>
<p style="color:black;">- Normalisados Numericos a int o float segun regla</p>    
<p style="color:black;">- Normalisados Fechas segun regla YYYY/MM/DD</p>    
<p style="color:black;">- Resguardo <code>datasets_salida/limpios/clientes_limpio.csv</code>.</p>
<p style="color:black;">- Resguardo <code>datasets_salida/limpios/marketing_limpio.csv</code>.</p>
<p style="color:black;">- Resguardo <code>datasets_salida/limpios/ventas_limpio.csv</code>.</p>
<p style="color:blue;">- Registros filtrados eliminados</p>
<p style="color:black;">- Resguardo <code>datasets_salida/reportes/reporte_limpieza.xlsx</code> con hojas (duplicados borrados, outliers, totales)</p>
</div>

In [ ]:
# 1. Crear df_duplicados_total
print (f"""
{cantidad_duplicados=}
""")
# Filtra las filas donde la columna 'problemas' contiene la subcadena 'duplicado_exacto'
df_duplicados_total = errores_df[errores_df['problemas'].str.contains('duplicado_exacto', case=False, na=False)].copy()

# 2. Crear df_outliers_total
# Filtra las filas donde la columna 'problemas' contiene la subcadena 'outlier_'
df_outliers_total = errores_df[errores_df['problemas'].str.contains('outlier_', case=False, na=False)].copy()

# 3. Crear df_resumen (Combinación y Ordenamiento)
# Concatena los dos DataFrames creados
df_resumen = pd.concat([df_duplicados_total, df_outliers_total])

# Ordena el DataFrame resultante por la columna 'problemas'
df_resumen = df_resumen.sort_values(by="problemas").reset_index(drop=True)
'''
print (f"""
df_duplicados_total
{df_duplicados_total}

{"-"*100}

df_outliers_total
{df_outliers_total}

{"-"*100}
df_resumen
{df_resumen}
{"-"*100}
""")
'''



In [ ]:
# Guardar en Excel con manejo de fallo si no existe el engine
try:
    with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
        try:
            df_duplicados_total.to_excel(writer, index=False, sheet_name='duplicados')
        except Exception:
            pd.DataFrame().to_excel(writer, index=False, sheet_name='duplicados')
        try:
            df_outliers_total.to_excel(writer, index=False, sheet_name='outliers')
        except Exception:
            pd.DataFrame().to_excel(writer, index=False, sheet_name='outliers')
        try:
            df_resumen.to_excel(writer, index=False, sheet_name='resumen_filtros')
        except Exception:
            pd.DataFrame(lista_resumen).to_excel(writer, index=False, sheet_name='resumen_filtros')
    guardado_ok = True
except Exception as e_openpyxl:
    mensajes.append('Error usando openpyxl: ' + str(e_openpyxl))
    try:
        with pd.ExcelWriter(ruta_excel) as writer:
            df_duplicados_total.to_excel(writer, index=False, sheet_name='duplicados')
            df_outliers_total.to_excel(writer, index=False, sheet_name='outliers')
            df_resumen.to_excel(writer, index=False, sheet_name='resumen_filtros')
        guardado_ok = True
    except Exception as e_default:
        mensajes.append('Error sin engine: ' + str(e_default))
        try:
            df_duplicados_total.to_csv(carpeta_reportes / 'duplicados.csv', index=False, encoding='utf-8')
            df_outliers_total.to_csv(carpeta_reportes / 'outliers.csv', index=False, encoding='utf-8')
            df_resumen.to_csv(carpeta_reportes / 'resumen_filtros.csv', index=False, encoding='utf-8')
            mensajes.append('Se guardaron CSVs separados como fallback.')
            guardado_ok = True
        except Exception as e_csv:
            mensajes.append('Error guardando CSV fallback: ' + str(e_csv))
            guardado_ok = False

print('Guardado OK:', guardado_ok)
if mensajes:
    print('Mensajes/Errores durante guardado:')
    for m in mensajes:
        print('-', m)
print('Ruta final esperada del Excel (si guardado):', ruta_excel)
print('Resumen por dataset:')
print(df_resumen)

ventas.csv  análisis de ventas, limpieza de datos y estadísticas descriptivas.
 	
clientes.csv  unirse a las ventas mediante el uso de funciones de combinación para analizar características de los clientes relacionados con sus 	compras.
 	
marketing.csv analizar la efectividad de las campañas de marketing en las ventas y buscar correlaciones.


In [ ]:

# 3) Tipos y limpiezas básicas
ventas['fecha_venta'] = pd.to_datetime(ventas['fecha_venta'], errors='coerce')
marketing['fecha_inicio'] = pd.to_datetime(marketing['fecha_inicio'], errors='coerce')
marketing['fecha_fin'] = pd.to_datetime(marketing['fecha_fin'], errors='coerce')

# Asegurar numéricos
ventas['precio'] = pd.to_numeric(ventas['precio'], errors='coerce')
ventas['cantidad'] = pd.to_numeric(ventas['cantidad'], errors='coerce')
clientes['ingresos'] = pd.to_numeric(clientes['ingresos'], errors='coerce')
marketing['costo'] = pd.to_numeric(marketing['costo'], errors='coerce')

# 4) Crear columnas útiles
ventas['monto'] = ventas['precio'] * ventas['cantidad']

# 5) Merge ejemplo: ventas + marketing por 'producto' (asignar canal a cada venta)
ventas_marketing = pd.merge(
    ventas,
    marketing[['producto', 'id_campanha', 'canal', 'costo', 'fecha_inicio', 'fecha_fin']],
    on='producto',
    how='left',   # left para conservar todas las ventas aunque no tengan campana asociada
    validate='m:1'  # opcional: espera muchos registros ventas para 1 campaña por producto
)

# 6) Agregados: ventas por canal
ventas_por_canal = (
    ventas_marketing
    .groupby('canal', dropna=False)
    .agg(
        total_monto=('monto', 'sum'),
        cantidad_transacciones=('monto', 'count'),
        ticket_promedio=('monto', 'mean')
    )
    .reset_index()
)

# 7) Agregado: ventas por categoria y canal
ventas_categoria_canal = (
    ventas_marketing
    .groupby(['categoria', 'canal'], dropna=False)
    .agg(
        total_monto     = ('monto', 'sum'),
        transacciones   = ('monto', 'count'),
        ticket_promedio = ('monto', 'mean')
    )
    .reset_index()
)

# 8) Guardar resultados (opcional)
ventas_por_canal.to_csv('/mnt/data/ventas_por_canal.csv', index=False)
ventas_categoria_canal.to_csv('/mnt/data/ventas_categoria_canal.csv', index=False)

# 9) ¿Y clientes? Si tienes id_cliente en ventas:
# ventas_con_clientes = ventas.merge(clientes, on='id_cliente', how='left')

# 10) Checks útiles
# - Ver duplicados en claves: ventas['id_venta'].duplicated().sum()
# - Ver clientes sin ventas: clientes[~clientes['id_cliente'].isin(ventas.get('id_cliente', []))]

Recomendaciones prácticas (breves, accionables)

Si querés unir ventas con clientes agregá id_cliente a ventas_limpio (registro en punto de venta o mapeo).

Revisá duplicados en producto dentro de marketing (puede haber varias campañas por producto: decidir estrategia — por ejemplo filtrar la campaña activa por fecha).

Elegí tipo de join con criterio pedagógico:

left join para preservar todas las ventas (evitar perder datos).

inner join si sólo te interesa el subset con campaña asociada.

Creá fecha de periodo (día/semana/mes) para series temporales: ventas['mes'] = ventas['fecha_venta'].dt.to_period('M').

Documentá supuestos: por qué usás how='left', cómo tratás ventas sin campaña, cómo imputás nulos en precio/cantidad.

Ejemplo aplicado (interpretación cotidiana)

Imaginá a Juan, vendedor en una pyme familiar con 2 hijos. Quiere saber si la campaña en Instagram está trayendo ventas: con el merge ventas + marketing por producto obtiene canal asignado a cada venta. Luego agrupa por canal y ve: “Instagram” tiene muchas visitas pero ticket promedio bajo — decisión: ajustar oferta o dirigir una campaña de cross-sell.

In [ ]:
a mano
ver que es lo que falta en drop na
precios == 0 buscar en categoria el producto o promedo si no hay otro dato
duplicate si el id es =


In [ ]:
productos mas vendidos 

In [ ]:
ventas por mes

In [ ]:
git remote set-url origin https://github.com/CursosAGT/nombre-nuevo.git
git push -u origin main
https://github.com/CursosAGT/-GarciaTrabaArielH-Comisi-n25262-TPI_Data_Analytics/blob/main/Garcia%20Traba%20Ariel%20H%20-%20Comisi%C3%B3n%2025262%20-%20TPI%20Data%20Analytics.ipynb
Garcia Traba Ariel H - Comisión 25262 - TPI Data Analytics.ipynb
https://github.com/CursosAGT/-GarciaTrabaArielH-Comisi-n25262-TPI_Data_Analytics/blob/main/datasets_entrada/clientes.csv
https://github.com/CursosAGT/-GarciaTrabaArielH-Comisi-n25262-TPI_Data_Analytics/blob/main/datasets_entrada/marketing.csv
https://github.com/CursosAGT/-GarciaTrabaArielH-Comisi-n25262-TPI_Data_Analytics/blob/main/datasets_entrada/ventas.csv.csv